In [1]:
import pandas as pd
import requests
import numpy as np
from lightweight_charts import Chart
from stock_indicators import indicators, Quote
from datetime import datetime, timedelta
import asyncio
import nest_asyncio

nest_asyncio.apply()

In [2]:
# Add required imports
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# import yfinance as yf
# df = yf.download('SPY', start='2010-01-01', multi_level_index=False)
# df.reset_index(inplace=True)
# df.to_csv('SPY.csv', index=False)
df = pd.read_csv('SPY.csv')
rawdf = df.copy()
df['Date'] = pd.to_datetime(df['Date'])
df.head()

,Date,Close,High,Low,Open,Volume
0,2010-01-04,85.279205,85.324353,83.909682,84.556820,118944600
1,2010-01-05,85.504936,85.542563,84.917998,85.226520,111579900
2,2010-01-06,85.565147,85.775842,85.354452,85.422173,116074400
3,2010-01-07,85.926346,86.031693,85.166334,85.407129,131091100
4,2010-01-08,86.212257,86.249884,85.527499,85.700567,126402800


In [4]:
quotes = [
    Quote(d, o, h, l, c, v)
    for d, o, h, l, c, v in zip(
        df['Date'],
        df['Open'],
        df['High'],
        df['Low'],
        df['Close'],
        df['Volume']
    )
]


In [5]:
# Calculate Supertrend
df['supertrend'] = [r.super_trend for r in indicators.get_super_trend(quotes, 14, 3)]
df['supertrend_direction'] = 0.0
df['supertrend_direction'] = np.where(df['supertrend'] > df['Close'], 0.0, 1.0)
df['crossover_supertrend'] = df['supertrend_direction'].diff()

In [6]:
# Convert supertrend to numeric for calculations
df['supertrend'] = pd.to_numeric(df['supertrend'], errors='coerce')

# Initialize columns for backtesting
df['Position'] = 0  # 1 for long, 0 for no position, -1 for short
df['Signal'] = df['crossover_supertrend']  # Buy (1) or Sell (-1) signals
df['Entry_Price'] = 0.0
df['Exit_Price'] = 0.0
df['Trade_Return'] = 0.0
df['Cumulative_Return'] = 0.0

# Initialize variables for tracking trades
current_position = 0
entry_price = 0
initial_capital = 100000  # Starting with $100,000
capital = initial_capital
trade_history = []

# Perform backtesting
for i in range(1, len(df)):
    signal = df.iloc[i]['Signal']
    current_price = df.iloc[i]['Close']
    
    # Buy Signal
    if signal == 1 and current_position == 0:
        current_position = 1
        entry_price = current_price
        df.loc[df.index[i], 'Position'] = 1
        df.loc[df.index[i], 'Entry_Price'] = entry_price
        trade_history.append({
            'Date': df.iloc[i]['Date'],
            'Type': 'Buy',
            'Price': entry_price,
            'Capital': capital
        })
    
    # Sell Signal
    elif signal == -1 and current_position == 1:
        exit_price = current_price
        returns = (exit_price - entry_price) / entry_price
        capital *= (1 + returns)
        df.loc[df.index[i], 'Position'] = 0
        df.loc[df.index[i], 'Exit_Price'] = exit_price
        df.loc[df.index[i], 'Trade_Return'] = returns
        current_position = 0
        trade_history.append({
            'Date': df.iloc[i]['Date'],
            'Type': 'Sell',
            'Price': exit_price,
            'Capital': capital
        })

    # Update cumulative returns
    df.loc[df.index[i], 'Cumulative_Return'] = (capital / initial_capital - 1) * 100

# Convert trade history to DataFrame
trade_df = pd.DataFrame(trade_history)

# Calculate performance metrics
total_trades = len(trade_df) // 2  # Divide by 2 since each complete trade has a buy and sell
winning_trades = len(df[df['Trade_Return'] > 0])
losing_trades = len(df[df['Trade_Return'] < 0])
win_rate = winning_trades / total_trades if total_trades > 0 else 0
final_return = (capital / initial_capital - 1) * 100
max_drawdown = (df['Cumulative_Return'].cummax() - df['Cumulative_Return']).max()

In [7]:
# Plot equity curve
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(rows=2, cols=1, shared_xaxes=True,
                   vertical_spacing=0.03, subplot_titles=('Price with Supertrend', 'Equity Curve'),
                   row_heights=[0.7, 0.3])

# Add candlestick
fig.add_trace(go.Candlestick(x=df['Date'],
                            open=df['Open'],
                            high=df['High'],
                            low=df['Low'],
                            close=df['Close'],
                            name='OHLC'),
              row=1, col=1)

# Add Supertrend line
fig.add_trace(go.Scatter(x=df['Date'],
                        y=df['supertrend'],
                        mode='lines',
                        name='Supertrend',
                        line=dict(color='yellow')),
              row=1, col=1)

# Add buy signals
buy_signals = df[df['crossover_supertrend'] == 1]
fig.add_trace(go.Scatter(x=buy_signals['Date'],
                        y=buy_signals['Low'],
                        mode='markers',
                        name='Buy Signal',
                        marker=dict(color='green', symbol='triangle-up', size=10)),
              row=1, col=1)

# Add sell signals
sell_signals = df[df['crossover_supertrend'] == -1]
fig.add_trace(go.Scatter(x=sell_signals['Date'],
                        y=sell_signals['High'],
                        mode='markers',
                        name='Sell Signal',
                        marker=dict(color='red', symbol='triangle-down', size=10)),
              row=1, col=1)

# Add equity curve
fig.add_trace(go.Scatter(x=df['Date'],
                        y=df['Cumulative_Return'],
                        mode='lines',
                        name='Equity Curve',
                        line=dict(color='blue')),
              row=2, col=1)

# Update layout
fig.update_layout(
    title_text="Supertrend Strategy Backtest Results",
    xaxis_rangeslider_visible=False,
    height=800
)

fig.show()

In [8]:
# Print performance metrics
print(f"Performance Metrics (2010-2025):")
print(f"--------------------------------")
print(f"Initial Capital: ${initial_capital:,.2f}")
print(f"Final Capital: ${capital:,.2f}")
print(f"Total Return: {final_return:.2f}%")
print(f"Total Trades: {total_trades}")
print(f"Winning Trades: {winning_trades}")
print(f"Losing Trades: {losing_trades}")
print(f"Win Rate: {win_rate:.2%}")
print(f"Maximum Drawdown: {max_drawdown:.2f}%")

Performance Metrics (2010-2025):
--------------------------------
Initial Capital: $100,000.00
Final Capital: $292,869.34
Total Return: 192.87%
Total Trades: 66
Winning Trades: 38
Losing Trades: 28
Win Rate: 57.58%
Maximum Drawdown: 30.08%


In [9]:
# Calculate Buy and Hold Returns
initial_price = df.iloc[0]['Close']
final_price = df.iloc[-1]['Close']
buy_hold_return = ((final_price - initial_price) / initial_price) * 100

print("\nBuy and Hold Strategy:")
print(f"--------------------------------")
print(f"Initial Price: ${initial_price:.2f}")
print(f"Final Price: ${final_price:.2f}")
print(f"Total Return: {buy_hold_return:.2f}%")
print(f"Final Capital with Buy & Hold: ${initial_capital * (1 + buy_hold_return/100):,.2f}")

# Compare with Supertrend Strategy
print(f"\nStrategy Comparison:")
print(f"--------------------------------")
print(f"Supertrend Strategy Return: {final_return:.2f}%")
print(f"Buy & Hold Return: {buy_hold_return:.2f}%")
print(f"Outperformance: {final_return - buy_hold_return:.2f}%")


Buy and Hold Strategy:
--------------------------------
Initial Price: $85.28
Final Price: $667.80
Total Return: 683.07%
Final Capital with Buy & Hold: $783,074.82

Strategy Comparison:
--------------------------------
Supertrend Strategy Return: 192.87%
Buy & Hold Return: 683.07%
Outperformance: -490.21%


# Parameter Optimization
Now let's optimize the Supertrend parameters by testing different combinations of periods and multipliers.

In [10]:
# Function to backtest Supertrend strategy with different parameters
def backtest_supertrend(df, period, multiplier):
    # Create quotes for the indicators
    quotes = [
        Quote(d, o, h, l, c, v)
        for d, o, h, l, c, v in zip(
            df['Date'],
            df['Open'],
            df['High'],
            df['Low'],
            df['Close'],
            df['Volume']
        )
    ]
    
    # Calculate Supertrend
    df_test = df.copy()
    df_test['supertrend'] = [r.super_trend for r in indicators.get_super_trend(quotes, period, multiplier)]
    df_test['supertrend_direction'] = 0.0
    df_test['supertrend_direction'] = np.where(df_test['supertrend'] > df_test['Close'], 0.0, 1.0)
    df_test['Signal'] = df_test['supertrend_direction'].diff()
    
    # Initialize variables
    capital = 100000
    current_position = 0
    entry_price = 0
    
    # Perform backtesting
    for i in range(1, len(df_test)):
        signal = df_test.iloc[i]['Signal']
        current_price = df_test.iloc[i]['Close']
        
        # Buy Signal
        if signal == 1 and current_position == 0:
            current_position = 1
            entry_price = current_price
        
        # Sell Signal
        elif signal == -1 and current_position == 1:
            exit_price = current_price
            returns = (exit_price - entry_price) / entry_price
            capital *= (1 + returns)
            current_position = 0
    
    final_return = (capital / 100000 - 1) * 100
    return final_return

In [11]:
# Grid search parameters
periods = range(5, 31, 2)  # Test periods from 5 to 30 in steps of 2
multipliers = [x/10 for x in range(10, 51, 5)]  # Test multipliers from 1 to 5 in steps of 0.5

print("Starting optimization...")
print(f"Testing {len(periods)} different periods")
print(f"Testing {len(multipliers)} different multipliers")
print(f"Total combinations to test: {len(periods) * len(multipliers)}")

# Store results
results = []

# Perform grid search with progress tracking
total_combinations = len(periods) * len(multipliers)
current = 0

for period in periods:
    for multiplier in multipliers:
        current += 1
        if current % 10 == 0:  # Show progress every 10 combinations
            print(f"Progress: {current}/{total_combinations} combinations tested")
            
        return_pct = backtest_supertrend(df, period, multiplier)
        results.append({
            'period': period,
            'multiplier': multiplier,
            'return': return_pct
        })
        
# Convert results to DataFrame
results_df = pd.DataFrame(results)

# Find best parameters
best_result = results_df.loc[results_df['return'].idxmax()]
print("\nOptimization Complete!")
print("\nBest Parameters Found:")
print(f"Period: {best_result['period']}")
print(f"Multiplier: {best_result['multiplier']}")
print(f"Return: {best_result['return']:.2f}%")

Starting optimization...
Testing 13 different periods
Testing 9 different multipliers
Total combinations to test: 117
Progress: 10/117 combinations tested
Progress: 20/117 combinations tested
Progress: 30/117 combinations tested
Progress: 40/117 combinations tested
Progress: 50/117 combinations tested
Progress: 60/117 combinations tested
Progress: 70/117 combinations tested
Progress: 80/117 combinations tested
Progress: 90/117 combinations tested
Progress: 100/117 combinations tested
Progress: 110/117 combinations tested

Optimization Complete!

Best Parameters Found:
Period: 5.0
Multiplier: 2.5
Return: 298.66%


In [12]:
# Visualize results with heatmap
# Pivot the results for the heatmap
heatmap_data = results_df.pivot(index='period', columns='multiplier', values='return')

# Create heatmap
plt.figure(figsize=(15, 10))
sns.heatmap(heatmap_data, 
            annot=True, 
            fmt='.1f', 
            cmap='RdYlGn',
            center=0,
            cbar_kws={'label': 'Return %'})
plt.title('Supertrend Strategy Returns by Parameters')
plt.xlabel('Multiplier')
plt.ylabel('Period')
plt.show()

# Update the strategy with optimal parameters
print("\nUpdating strategy with optimal parameters...")
df['supertrend'] = [r.super_trend for r in indicators.get_super_trend(quotes, int(best_result['period']), best_result['multiplier'])]
df['supertrend_direction'] = 0.0
df['supertrend_direction'] = np.where(df['supertrend'] > df['Close'], 0.0, 1.0)
df['crossover_supertrend'] = df['supertrend_direction'].diff()


Updating strategy with optimal parameters...


C:\Users\jwang\AppData\Local\Temp\ipykernel_23736\3624419931.py:16: UserWarning:

FigureCanvasAgg is non-interactive, and thus cannot be shown



In [13]:
df['supertrend'] = df['supertrend'].astype(str)

# Now you can serialize your DataFrame
df.to_json() 

'{"Date":{"0":1262563200000,"1":1262649600000,"2":1262736000000,"3":1262822400000,"4":1262908800000,"5":1263168000000,"6":1263254400000,"7":1263340800000,"8":1263427200000,"9":1263513600000,"10":1263859200000,"11":1263945600000,"12":1264032000000,"13":1264118400000,"14":1264377600000,"15":1264464000000,"16":1264550400000,"17":1264636800000,"18":1264723200000,"19":1264982400000,"20":1265068800000,"21":1265155200000,"22":1265241600000,"23":1265328000000,"24":1265587200000,"25":1265673600000,"26":1265760000000,"27":1265846400000,"28":1265932800000,"29":1266278400000,"30":1266364800000,"31":1266451200000,"32":1266537600000,"33":1266796800000,"34":1266883200000,"35":1266969600000,"36":1267056000000,"37":1267142400000,"38":1267401600000,"39":1267488000000,"40":1267574400000,"41":1267660800000,"42":1267747200000,"43":1268006400000,"44":1268092800000,"45":1268179200000,"46":1268265600000,"47":1268352000000,"48":1268611200000,"49":1268697600000,"50":1268784000000,"51":1268870400000,"52":1268956

In [14]:
# Supertrend
if __name__ == '__main__':

    chart = Chart(title="Supertrend", maximize=True)
    chart.legend(visible=True, color_based_on_candle=True)
    # chart.layout(background_color="white")

    # Set the main candlestick data for the chart.
    # The 'lightweight-charts' library expects a DataFrame with columns like 'Date', 'Open', 'High', 'Low', 'Close'.
    chart.set(df)

    # Create line series for Supertrend

    supertrend_line = chart.create_line('supertrend', color='#ffeb3b', width=1, price_line=False, price_label=False)
    supertrend_line.set(df[['Date', 'supertrend']])

    # Initialize a list to hold the markers
    markers = []

    # Iterate through the DataFrame to find crossover points
    for i in range(1, len(df)):

        supertrend_diff = df.iloc[i]['crossover_supertrend']
        
        current_time = df.iloc[i]['Date']

        # Check for buy signal (supertrend crossover up df['Close'])
        if supertrend_diff == 1 :
            markers.append({
                'time': current_time,
                'position': 'below',
                'shape': 'arrow_up',
                'color': '#33de3d',
                'text': 'Buy'
            })
        
        # Check for sell signal (supertrend crossove down df['Close'])
        elif supertrend_diff == -1 :
            markers.append({
                'time': current_time,
                'position': 'above',
                'shape': 'arrow_down',
                'color': '#f485fb',
                'text': 'Sell'
            })

    # Add all markers at once. It's more efficient than adding them individually in a loop.
    if markers:
        chart.marker_list(markers)

chart.show(block=True)